# Installs and Imports

In [0]:
# install dependencies:
# (use +cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

In [0]:
# install detectron2:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

fatal: destination path 'detectron2_repo' already exists and is not an empty directory.
Obtaining file:///content/detectron2_repo
  Found existing installation: detectron2 0.1
    Can't uninstall 'detectron2'. No files were found to uninstall.
  Running setup.py develop for detectron2


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

import cv2
import os
import numpy as np
import json
from detectron2.structures import BoxMode

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Dataset

Construct Dictionary of SharkIDs to Classes

In [0]:
def createSharkClassID(filename):
  json_file = filename

  SharkClassDictionary = {}

  # Open the file
  with open(json_file) as f:
    img_annotations = json.load(f)

  classID = 0
  for i, values in enumerate(img_annotations):
    # Get the sharkID out
    sharkID   = values["id"]

    # If the current sharkID has not been assigned a classID
    if(sharkID not in SharkClassDictionary):
      # Add the sharkID to the dictionary, giving it a classID
      SharkClassDictionary[sharkID] = classID
      # Increment the classID for the next sharkID
      classID = classID + 1

  return SharkClassDictionary

# Generate dictionary
# SharkClassDictionary = createSharkClassID("data/train/data.json")
SharkClassDictionary = createSharkClassID("/content/drive/My Drive/sharkdata/all_data/train/data.json")
# SharkClassDictionary = createSharkClassID("/content/drive/My Drive/sharkdata/train/data.json")
# print(SharkClassDictionary)
# print(list(SharkClassDictionary.keys()))

Create Class List

In [0]:
ClassList = [""] * len(SharkClassDictionary)
for sharkID in list(SharkClassDictionary):
  ClassList[SharkClassDictionary[sharkID]] = sharkID

Registering the shark data with detectron2

In [0]:
# Construct the shark dictionaries
# This function will return the items in our dataset
def constructSharkDicts(dataDirectory):
  print("Constructing shark dicts")

  # Find the json file in our file structure
  json_file = os.path.join(dataDirectory, "data.json")

  # Open the file, referring to it as f
  with open(json_file) as f:
    # Get the image annotations out using json to read the file containin the JSON object
    img_annotations = json.load(f)

  # List of dictionaries
  dataset_dicts = []
  
  # Loop over the image annotation values, using idx as the indexer and v to be the value
  for i, values in enumerate(img_annotations):
    # Print every 100 iteration for time tracking
    if(i % 1000 == 0): print(i)

    # if(i == 5000): break

    # Create empty record
    record = {}

    # Extract information from json file
    imageID   = values["imId"]
    filename  = dataDirectory + "/" + values["imId"] +".jpg"
    sharkID   = values["id"]
    sharkSide = values["side"]
    ymin, xmin, ymax, xmax = values["box_ymin_xmin_ymax_xmax"]

    # Check image exists (might not because of how I'm handling dataset)
    if(not os.path.isfile(filename)): continue

    # Get image height and width
    imHeight, imWidth = cv2.imread(filename).shape[:2]

    # Construct annotation object (bounding box)
    objs = []
    
    #Get ClassID from SharkID
    classID = SharkClassDictionary[sharkID]

    # Construct an annotation object (only one in this case)
    obj = \
    {
        "bbox": [xmin,ymin,xmax,ymax],
        "bbox_mode": BoxMode.XYXY_ABS,
        "category_id": classID, #this needs to be an INTEGER ID for the shark's category, so we have created a mapping from sharkIDs to classIDs
        "iscrowd": 0
    }
    objs.append(obj)

    # Populate record
    record["file_name"] = filename
    record["image_id"]  = imageID
    record["height"]    = imHeight
    record["width"]     = imWidth
    record["annotations"] = objs

    # Append this record to the list of dictionaries
    dataset_dicts.append(record)
    
    # Only do it for 3 images (time saving)
    # print()
    # if i == 2:
      # break

  # Return the list of dictionaries
  return dataset_dicts

tmpDict = constructSharkDicts("/content/drive/My Drive/sharkdata/all_data/train/")
torch.save(tmpDict,"/content/drive/My Drive/sharkdata/all_data/train/sharkTrainDicts.pt")

tmpDict = constructSharkDicts("/content/drive/My Drive/sharkdata/all_data/val/")
torch.save(tmpDict,"/content/drive/My Drive/sharkdata/all_data/val/sharkValDicts.pt")

Constructing shark dicts
0
1000
2000
3000


KeyboardInterrupt: ignored

In [0]:
# # This function will return the items in our dataset
# def getSharkDicts(dataDirectory):
#   dataset_dicts = torch.load(dataDirectory)
#   return dataset_dicts


def getSharkTrainDicts():
  return torch.load("/content/drive/My Drive/sharkdata/all_data/train/sharkTrainDicts.pt")

def getSharkValDicts():
  return torch.load("/content/drive/My Drive/sharkdata/all_data/val/sharkValDicts.pt")



  # print("Getting shark dicts")

  # # Find the json file in our file structure
  # json_file = os.path.join(dataDirectory, "data.json")

  # # Open the file, referring to it as f
  # with open(json_file) as f:
  #   # Get the image annotations out using json to read the file containin the JSON object
  #   img_annotations = json.load(f)

  # # List of dictionaries
  # dataset_dicts = []
  
  # # Loop over the image annotation values, using idx as the indexer and v to be the value
  # for i, values in enumerate(img_annotations):
  #   # Print every 100 iteration for time tracking
  #   if(i % 1000 == 0): print(i)

  #   # if(i == 5000): break

  #   # Create empty record
  #   record = {}

  #   # Extract information from json file
  #   imageID   = values["imId"]
  #   filename  = dataDirectory + "/" + values["imId"] +".jpg"
  #   sharkID   = values["id"]
  #   sharkSide = values["side"]
  #   ymin, xmin, ymax, xmax = values["box_ymin_xmin_ymax_xmax"]

  #   # Check image exists (might not because of how I'm handling dataset)
  #   if(not os.path.isfile(filename)): continue

  #   # Get image height and width
  #   imHeight, imWidth = cv2.imread(filename).shape[:2]

  #   # Construct annotation object (bounding box)
  #   objs = []
    
  #   #Get ClassID from SharkID
  #   classID = SharkClassDictionary[sharkID]

  #   # Construct an annotation object (only one in this case)
  #   obj = \
  #   {
  #       "bbox": [xmin,ymin,xmax,ymax],
  #       "bbox_mode": BoxMode.XYXY_ABS,
  #       "category_id": classID, #this needs to be an INTEGER ID for the shark's category, so we have created a mapping from sharkIDs to classIDs
  #       "iscrowd": 0
  #   }
  #   objs.append(obj)

  #   # Populate record
  #   record["file_name"] = filename
  #   record["image_id"]  = imageID
  #   record["height"]    = imHeight
  #   record["width"]     = imWidth
  #   record["annotations"] = objs

  #   # Append this record to the list of dictionaries
  #   dataset_dicts.append(record)
    
  #   # Only do it for 3 images (time saving)
  #   # print()
  #   # if i == 2:
  #     # break

  # # Return the list of dictionaries
  # return dataset_dicts

In [0]:
# Enter into dataset and metadata catalogues
from detectron2.data import DatasetCatalog, MetadataCatalog
# for d in ["train", "val"]:
#   # Register shark_dictionary
#   # DatasetCatalog.register("shark_" + d, lambda d=d: getSharkDicts("data/" + d))
#   # DatasetCatalog.register("shark_" + d, lambda d=d: getSharkDicts("/content/drive/My Drive/sharkdata/all_data/" + d))
#   DatasetCatalog.register("shark_" + d, lambda d=d: getSharkDicts("/dicts/" + d))
#   # DatasetCatalog.register("shark_" + d, lambda d=d: getSharkDicts("/content/drive/My Drive/sharkdata/" + d))
#   # MetadataCatalog.get("shark_" + d).set(thing_classes=list(SharkClassDictionary.keys()))
#   MetadataCatalog.get("shark_" + d).set(thing_classes=ClassList)

# DatasetCatalog.register("shark_train", getSharkDicts("/content/drive/My Drive/sharkdata/all_data/train"))
DatasetCatalog.register("shark_train", getSharkTrainDicts())
MetadataCatalog.get("shark_train").set(thing_classes=ClassList)
# DatasetCatalog.register("shark_val", getSharkDicts("/content/drive/My Drive/sharkdata/all_data/val"))
DatasetCatalog.register("shark_val", getSharkValDicts())
MetadataCatalog.get("shark_val").set(thing_classes=ClassList)

shark_metadata = MetadataCatalog.get("shark_train")

ModuleNotFoundError: ignored

Visualise to check this worked



In [0]:
# dataset_dicts = getSharkDicts("/content/drive/My Drive/sharkdata/train")
dataset_dicts = getSharkTrainDicts() #getSharkDicts("/content/drive/My Drive/sharkdata/all_data/train")
print("Done")
for dictionary in random.sample(dataset_dicts, 3):
# for dictionary in dataset_dicts:
  img = cv2.imread(dictionary["file_name"])
  visualizer = Visualizer(img[:, :, ::-1], metadata=shark_metadata, scale=0.2)
  vis = visualizer.draw_dataset_dict(dictionary)
  # Print the class ID
  classID = ((dictionary["annotations"])[0])["category_id"]
  print(ClassList[classID])

  cv2_imshow(vis.get_image()[:, :, ::-1])

Getting shark dicts
0


KeyboardInterrupt: ignored

# Training

Training time

In [0]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

# COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml
# COCO-Detection/faster_rcnn_R_50_C4_1x.yaml
# COCO-Detection/retinanet_R_50_FPN_1x.yaml
# COCO-Detection/retinanet_R_50_FPN_3x.yaml
# COCO-Detection/retinanet_R_101_FPN_3x.yaml
modelLink = Detection/retinanet_R_101_FPN_3x.yaml

# default configuration
cfg = get_cfg()

# get the pretrained retinanet model
cfg.merge_from_file(model_zoo.get_config_file(modelLink))

# list of the dataset names for training (registered in datasetcatalog (?))
cfg.DATASETS.TRAIN = ("shark_train",)
# list of the dataset names for testing (registered in datasetcatalog (?))
cfg.DATASETS.TEST = ()

# number of data loading threads
cfg.DATALOADER.NUM_WORKERS = 2

# locate the pretrained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(modelLink)  # Let training initialize from model zoo

# number of images per batch
cfg.SOLVER.IMS_PER_BATCH = 2

# learning rate
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR

# max iterations
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
# cfg.SOLVER.MAX_ITER = 500    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset

# Minibatch size PER image - number of regions of interest (ROIs)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)

# Number of classes
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)
cfg.MODEL.RETINANET.NUM_CLASSES = len(SharkClassDictionary)  # only has one class (ballon)

# directories (?)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Get the default trainer:
## 1) Create model, optimiser, scheduler, dataloader from the given config
## 2) Load a checkpoint or cfg.MODEL.WEIGHTS if it exists
## 3) Register a few common hooks (?)
## This simplifies the standard model training workflow, so you don't have to write boilerplate code
trainer = DefaultTrainer(cfg)
# If true, and the last checkpoint exists, resume from it
# If false, load a model specified by the config
trainer.resume_or_load(resume=False)
trainer.train()

Loading config /content/detectron2_repo/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[01/28 22:12:57 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

In [0]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

# Inference and Evaluation

Inference and Evaluation

In [0]:
# Inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.6
# cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.25
cfg.DATASETS.TEST = ("shark_val", )
# cfg.DATASETS.TEST = ("shark_train", )
# Create a simple end-to-end predictor with the given config
## This predictor takes care of model loading and input preprocessing for you
predictor = DefaultPredictor(cfg)

In [0]:
# Visualise:
from detectron2.utils.visualizer import ColorMode
# dataset_dicts = getSharkDicts("/content/drive/My Drive/sharkdata/train")
dataset_dicts = getSharkValDicts() #getSharkDicts("/content/drive/My Drive/sharkdata/all_data/val")
# for dictionary in random.sample(dataset_dicts, 12):
for dictionary in random.sample(dataset_dicts, 12):
  im = cv2.imread(dictionary["file_name"])
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                  metadata=shark_metadata, 
                  scale=0.2, 
                #  instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
  )
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

  # Print the class ID
  classID = ((dictionary["annotations"])[0])["category_id"]
  print(ClassList[classID])

  img = v.get_image()[:, :, ::-1]
  # cv2_imshow(img)
  
  filename = "inferenceOutputs/" + dictionary["id-"] + dictionary["file_name"]
  cv2.imwrite(filename, img)

In [0]:
# AP
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("shark_val", cfg, False, output_dir="./output/")
# evaluator = COCOEvaluator("shark_train", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "shark_val")
# val_loader = build_detection_test_loader(cfg, "shark_train")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test